[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openvinotoolkit/openvino_tensorflow/blob/master/docs/cloud_instructions/Colab_OpenVINO_TensorFlow_classification_example.ipynb)

## INSTALL OpenVINO™ integration with TensorFlow


In [ ]:
# Upload the required wheel files, models and images in a google drive folder
# Uncomment and run the below command to copy them in your current workspace
#!cp /content/drive/MyDrive/TF-OV/working_dir_files/* . 

!pip -q install --upgrade pip
!pip -q install pillow

# Install stock TensorFlow
!pip -q install tensorflow==2.5.1 

# Install OpenVINO™ integration with TensorFlow
!pip -q install openvino-tensorflow

In [ ]:
!ldd --version
!python3 --version

In [ ]:
!git clone --quiet https://github.com/openvinotoolkit/openvino_tensorflow.git
%cd openvino_tensorflow
!git submodule init
!git submodule update --recursive
%cd ..


# Now lets infer using InceptionV3 pretained classification model

In [35]:
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import HTML

import os
import numpy as np
import tensorflow as tf
import openvino_tensorflow as ovtf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import time
import cv2

from common.utils import get_input_mode

In [36]:
def preprocess_image(file_name,
               input_height=299,
               input_width=299):
    """Reads input image file, resize it to given input height and width.
       Normalize the image using input mean and standard deviation and returns the image tensor
    """  
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img)
    resized_image = pil_img.resize((input_width, input_height))
    img_arr = image.img_to_array(resized_image)
    result = preprocess_input(img_arr[tf.newaxis,...])

    return result

In [37]:
def load_labels(label_file):
    """Parses the label file, assuming that labels are separated with a newline
       in the file and returns the list of labels.
    """  
    label = []
    proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

In [38]:
def infer_openvino_tensorflow(model_file, file_name , input_height, input_width, input_mean, input_std, label_file):
    """Takes the tensorflow model and all other input parameters as arguments. 
       Run inference on the classification model and prints the predictions.
    """
    print("CREATE MODEL - BEGIN")
    if model_file=="":
      model = InceptionV3(weights='imagenet')
    else:
      model = load_model(model_file)
    print("CREATE MODEL - END")

    print("PREDICTION - BEGIN") 

    img = preprocess_image(
    file_name,
    input_height=input_height,
    input_width=input_width)

    # Warmup
    results = model.predict(img)
    # Run
    
    for num_times in range(10):
        start = time.time()
        results = model.predict(img)
        elapsed = time.time() - start
        print('Inference time in ms: %f' % (elapsed * 1000))
            
    print("PREDICTION - END")
    results = np.squeeze(results)

    if label_file:
        top_k = results.argsort()[-5:][::-1]
        labels = load_labels(label_file)
        for i in top_k:
            print(labels[i+1], results[i])
    else:
        print("No label file provided. Cannot print classification results")

In [ ]:
file_name = tf.keras.utils.get_file(
    'grace_hopper.jpg',
    "https://www.tensorflow.org/images/grace_hopper.jpg")
model_file = ""
label_file = tf.keras.utils.get_file(
    'ImageNetLabels.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
input_height = 299
input_width = 299
input_mean = 0
input_std = 255
backend_name = "CPU"
    

#Print list of available backends
print('Available Backends:')
backends_list = ovtf.list_backends()
for backend in backends_list:
    print(backend)
ovtf.set_backend(backend_name)

print("OpenVINO TensorFlow is enabled")
infer_openvino_tensorflow(model_file, file_name, input_height, input_width, input_mean, input_std, label_file)

In [ ]:
#Disable
ovtf.disable()
print("")
print("")
print("OpenVINO TensorFlow is disabled")
infer_openvino_tensorflow(model_file, file_name, input_height, input_width, input_mean, input_std, label_file )
ovtf.enable()
